In [ ]:

import requests 
from bs4 import BeautifulSoup
from pprint import pp
URL = 'https://books.toscrape.com/'
def explora_categoria(url):
    """ A partir de la URL de la página principal de una categoría, devuelve el nombre
        de la categoría y el número de libros """
    category_request=requests.get(url)
    category_request.encoding = category_request.apparent_encoding  # Corregimos el encoding
    html = category_request.text
    soup = BeautifulSoup(html, 'html.parser')
    etiquets=soup.find_all("strong")
    return (etiquets[0].text,int(etiquets[1].text))

def categorias():
    """ Devuelve un conjunto de parejas (nombre, número libros) de todas las categorías """
    r = requests.get(URL)
    r.encoding = r.apparent_encoding  # Corregimos el encoding
    html = r.text
    soup = BeautifulSoup(html, 'html.parser')
    #https://www.crummy.com/software/BeautifulSoup/bs4/doc/#the-string-argument:~:text=soup.find_all(%22a%22%2C%20attrs%3D%7B%22class%22%3A%20%22sister%22%7D)
    etiquetas = soup.find_all("div",{'class':"side_categories"})
    enlaces=etiquetas[0].find_all("a")
    return {explora_categoria(URL+enlace.attrs["href"])for enlace in enlaces[1:]}
    


con=categorias()
for c in con:
    print(c)


In [6]:
# APARTADO 2 #
import requests 
from bs4 import BeautifulSoup
URL = 'https://books.toscrape.com/'
def url_categoria(nombre):
    """ Devuelve la URL de la página principal de una categoría a partir de su nombre (ignorar
        espacios al principio y final y también diferencias en mayúsculas/minúsculas) """
    r = requests.get(URL)
    r.encoding = r.apparent_encoding  # Corregimos el encoding
    html = r.text
    soup = BeautifulSoup(html, 'html.parser')
    #https://www.geeksforgeeks.org/beautifulsoup-search-by-text-inside-a-tag/
    etiquetas = soup.find_all(lambda tag: tag.name=="a" and tag.text.strip().lower()==nombre.strip().lower())
    
    if(len(etiquetas)==0):
        return None
    else:
        return URL+etiquetas[0].attrs["href"]

print(url_categoria('Historical fiction'))

#https://ellibrodepython.com/switch-python hemos visto que esto es mas eficiente que if else    
def number(texto):
    numeros={
        'one':1,
        'two':2,
        'three':3,
        'four':4,
        'five':5
    }
    return numeros.get(texto.lower(),None)
def libros_de_la_pagina(url):
    """
    Devuelve un conjunto con todos los libros que se ven 
    """
    if url is None:
        return None
    r = requests.get(url)
    r.encoding = r.apparent_encoding  # Corregimos el encoding
    html = r.text
    soup = BeautifulSoup(html, 'html.parser')
    etiquetas=soup.find_all("article",{'class':"product_pod"})
    conjunto=set()
    for etiqueta in etiquetas:
        p=etiqueta.find("p")
        precio=etiqueta.find("p",{'class':"price_color"}).text[1:]
        titulo=etiqueta.find("h3").find("a").attrs["title"]
        c=(titulo,float(precio),number(p.attrs["class"][1]))
        conjunto.add(c)
    for c in conjunto:
        print(c)
    return conjunto
libros_de_la_pagina(url_categoria('Historical fiction'))


def todas_las_paginas(url):
    """ Sigue la paginación recopilando todas las URL *absolutas* atravesadas """
    

def libros_categoria(nombre):
    """ Dado el nombre de una categoría, devuelve un conjunto de tuplas 
        (titulo, precio, valoracion), donde el precio será un número real y la 
        valoración un número natural """
    ...



https://books.toscrape.com/catalogue/category/books/historical-fiction_4/index.html
('Voyager (Outlander #3)', 21.07, 5)
('Girl in the Blue Coat', 46.83, 2)
('The Red Tent', 35.66, 5)
('The Marriage of Opposites', 28.08, 4)
('The Guernsey Literary and Potato Peel Pie Society', 49.53, 1)
('The House by the Lake', 36.95, 1)
('Girl With a Pearl Earring', 26.77, 1)
('Glory over Everything: Beyond The Kitchen House', 45.84, 3)
('Forever and Forever: The Courtship of Henry Longfellow and Fanny Appleton', 29.69, 3)
('A Flight of Arrows (The Pathfinders #2)', 55.53, 5)
('Lilac Girls', 17.28, 2)
('Love, Lies and Spies', 20.55, 2)
('The Constant Princess (The Tudor Court #1)', 16.62, 3)
('The Last Painting of Sara de Vos', 55.55, 2)
('World Without End (The Pillars of the Earth #2)', 32.97, 4)
('Tipping the Velvet', 53.74, 1)
('A Paris Apartment', 39.01, 4)
('The Invention of Wings', 37.34, 1)
('Mrs. Houdini', 30.25, 5)
('The Passion of Dolssa', 28.32, 5)
